In [29]:
import torch

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
a = torch.arange(12).reshape(2,3,2)
b = torch.arange(6).reshape(2,3)
print(a)
print()
print(b)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]]])

tensor([[0, 1, 2],
        [3, 4, 5]])
